In [ ]:
%%capture

# For the demonstration, we need torchaudio and matplotlib to process example audios and visualize the spectrogram
import sys
!{sys.executable} -m pip install torchaudio soundfile matplotlib scipy

In [ ]:
import os
print(os.path.exists(r"C:\Users\x\AppData\Local\Programs\Python\Python313\Lib\site-packages\audioseal"))


In [ ]:
import torch
import torchaudio
import urllib

def download_sample_audio():
    url = "https://keithito.com/LJ-Speech-Dataset/LJ025-0076.wav"
    with open("test.wav", "wb") as f:
        resp = urllib.request.urlopen(url)
        f.write(resp.read())
    
    wav, sample_rate = torchaudio.load("test.wav")
    return wav, sample_rate

audio, sr = download_sample_audio()

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,3)

from notebook import play_audio, plot_waveform_and_specgram

plot_waveform_and_specgram(audio, sr, title="Original audio")

In [ ]:
play_audio(audio, sr)

In [ ]:
# Initialise Audioseal models
from audioseal import AudioSeal

# Load models
generator = AudioSeal.load_generator("audioseal_wm_16bits")
detector = AudioSeal.load_detector(("audioseal_detector_16bits"))


In [ ]:
# Generate 4 UNIQUE messages (16-bit)
messages = [torch.randint(0, 2, (1, 16)) for _ in range(4)]

for i, msg in enumerate(messages):
    print(f"Message {i+1}: {msg.numpy().flatten()}")  # Verify uniqueness

# Initialize variables
sr = 16000
original_audio = torch.randn(1, 1, sr)  # Test audio (1s @16kHz)
watermarked_audio = original_audio.clone()
alpha = 0.25  # Reduced strength for multiple embeddings  

# # If loading real audio (e.g., from torchaudio):
# audio, sr = torchaudio.load("test.wav")
# audio = audio.unsqueeze(0)  # Add batch dimension (if missing)
# audio = audio.unsqueeze(1)  # Add channel dimension (if mono)

# Embed watermarks sequentially
for idx, msg in enumerate(messages):
    # Generate watermark for current message
    watermark = generator.get_watermark(
        watermarked_audio, 
        sample_rate=sr,
        message=msg
    )

    # Apply watermark with reduced strength
    watermarked_audio = watermarked_audio + alpha * watermark
    
    # Calculate metrics after EACH embedding
    noise = watermarked_audio - original_audio
    snr = 10 * torch.log10(original_audio.pow(2).mean() / noise.pow(2).mean())
    
    print(f"\nAfter Watermark {idx+1}:")
    print("-" * 50)
    print(f"  SNR: {snr:.2f} dB")

    # Detect ALL previous watermarks
    for detect_idx in range(idx+1):
        detector.message = messages[detect_idx]  # Critical: Reset detector state
        prob, detected_msg = detector.detect_watermark(watermarked_audio,sample_rate=sr, message_threshold=0.5)
        ber = (messages[detect_idx] != detected_msg.round()).float().mean()
        print(f"  Watermark {detect_idx+1}: BER={ber:.2f}, Prob={prob:.2f}")


In [ ]:
# Calculate SNR
noise = watermarked_audio - original_audio
snr = 10 * torch.log10(original_audio.pow(2).mean() / noise.pow(2).mean())
print(f"Final SNR: {snr:.2f} dB")

In [ ]:
plot_waveform_and_specgram(watermarked_audio.squeeze(), sr, title="Multi-Watermarked Audio")
play_audio(watermarked_audio, sr)

In [ ]:
# #Detect watermarks
# def detect_watermarks(watermarked_audio, messages):
#     results = []
#     for msg in messages:
#         detector.message = msg  # Force detector to look for this message
#         prob, decoded_msg = detector.detect_watermark(watermarked_audio, sample_rate=sr)
#         ber = (msg != decoded_msg.round()).float().mean()
#         results.append((prob, ber))
#     return results

In [ ]:
#apply attacks
from attacks1 import AudioEffects1 as attacks1
# Apply MP3 compression
attacked_audio = attacks.mp3_compression(watermarked_audio, bitrate=64)

# Detect after attack
attacked_results = detect_watermarks(attacked_audio, messages)
                                    
print("Compressed audio shape:", compressed_audio.shape)
                                     
print("\n--- After MP3 Compression (64 kbps) ---")
for i, (prob, ber) in enumerate(attacked_results):
    print(f"Watermark {i+1}: Detection Prob = {prob:.2f}, BER = {ber:.2f}")

In [ ]:
plot_waveform_and_specgram(watermarked_audio.squeeze(), sr, title="Multi-Watermarked Audio")
play_audio(watermarked_audio, sr)